<a href="https://colab.research.google.com/github/Arun-nexus/nexux/blob/main/nexrt_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()  # upload your kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
! kaggle datasets download -d vinayaks0n1/imdb-tv-show-reviews

In [ ]:
import zipfile
with zipfile.ZipFile("imdb-tv-show-reviews.zip","r") as zip_ref:
  zip_ref.extractall("dataset")

In [ ]:
dataset="dataset"

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [ ]:
df=pd.read_csv("/content/dataset/imdb_tvshows.csv")

In [ ]:
df.head()

In [ ]:
def clean(text):
  text=str(text).lower()
  text=re.sub(r"[^a-zA-Z\s]","",text)
  return text
df["Review"]=df["Review"].apply(clean)

In [ ]:
data=" ".join(df["Review"].to_list())

In [ ]:
token=Tokenizer(num_words=10000)
token.fit_on_texts([data])
total_words=10000

In [ ]:
tokens=token.texts_to_sequences([data])[0]
tokens

In [ ]:
from collections.abc import Sequence
input_sequence=[]

for i in range(1,total_words):
  Sequence=tokens[:i+1]
  input_sequence.append(Sequence)


In [ ]:
total_words

In [ ]:
lenght=max([len(seq) for seq in input_sequence])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
pad_sequence=pad_sequences(input_sequence,maxlen=lenght,padding="pre")

In [ ]:
pad_sequence

In [ ]:
x=pad_sequence[:,:-1]
y=pad_sequence[:,-1]

In [ ]:
x

In [ ]:
y

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y=to_categorical(y,num_classes=total_words)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
x.shape

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=total_words,output_dim=64))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))



In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.build(input_shape=(None,x.shape[1]))
model.summary()

In [ ]:
stopper=EarlyStopping(
    patience=3,
    monitor="val_accuracy",
    verbose=1,
    restore_best_weights=True
)

In [ ]:
hist=model.fit(x,y,epochs=100,callbacks=stopper,validation_split=0.3)